In [43]:
# import arrow
# import numpy as np
# import glob



# def get_windar(start,end):
#     start_run = arrow.get(start)
#     end_run = arrow.get(end)
#     thres = arrow.get('2014-09-12')

#     arrow_array = []
#     ncar = []

#     for r in arrow.Arrow.span_range('day', start_run, end_run):
#         arrow_array.append(r)

#     dayslen = len(arrow_array)
#     #print(dayslen)
#     ts = np.zeros(dayslen)
#     #print(arrow_array)

#     # ops_y2014m09d12.nc

#     for i in range(0,dayslen):

#         tdate = arrow_array[i][0]
#         yy = tdate.format('YYYY')
#         mm = tdate.format('MM')
#         dd = tdate.format('DD')
#         ymd = f'y{yy}m{mm}d{dd}'

#         if tdate < thres:
#             dirstring = '/results/forcing/atmospheric/GEM2.5/gemlam/'
#             guess = 'gemlam_'+ ymd +'.nc' 
#         else:
#             dirstring = '/results/forcing/atmospheric/GEM2.5/operational/'
#             guess = 'ops_'+ ymd +'.nc'

#         #tdir = '/data/tjarniko/results/RIV_PIL/BASE/ncs/'

#         w = glob.glob(dirstring+guess)
#         w = w[0]
#         ncar.append(w)
    
#     return ncar

# start ='2013-01-01'; end ='2013-12-31'
# ncar_2013 = get_windar(start,end)
# start ='2014-01-01'; end ='2014-12-31'
# ncar_2014 = get_windar(start,end)
# start ='2015-01-01'; end ='2015-12-31'
# ncar_2015 = get_windar(start,end)
# start ='2016-01-01'; end ='2016-12-31'
# ncar_2016 = get_windar(start,end)

In [44]:
start ='2013-01-01'; end ='2013-12-31'

def get_dates(start,end):
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    thres = arrow.get('2014-09-12')

    arrow_array = []
    ncar = []

    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    print(dayslen)
    #print(dayslen)
    ts = np.zeros(dayslen)
    #print(arrow_array)

    # ops_y2014m09d12.nc

    for i in range(0,dayslen):

        tdate = arrow_array[i][0]
        tdate = arrow_array[i][0]
        yy = tdate.format('YYYYMMDD')

        ncar.append(yy)
        
    return ncar
        
dar2013 = get_dates(start,end)


365


In [45]:
def weights_intrp_mtx_ops(odata):
    '''POINT: interpolates data from the atmospheric grid onto the ocean grid using a weights matrix | CALLS: | NOTES: this is much faster than interpolating with scipy! | TAKES : one array of size lats 266 x lons 256 of atmospheric data | GIVES: an array of NEMO-Salish size of the new data interpolated | USAGE: | ROLE IN CLUSTER PROJECT: | written by Michael Dunphy, adapted by tjšj, 2017'''
    from IPython import embed
    import netCDF4 as nc
    import scipy.interpolate as spi
    import scipy.sparse as sp
    import numpy as np
    '''takes a matrix of size lon(256), lat(266) and interpolates it onnto the NEMO gird
    size is 266 x 256'''
    weightsfile = '/home/mdunphy/MEOPAR/NEMO-forcing/grid/weights-gem2.5-ops.nc'
    with nc.Dataset(weightsfile) as f:
        s1 = f.variables['src01'][:]-1  # minus one for fortran-to-python indexing
        s2 = f.variables['src02'][:]-1
        s3 = f.variables['src03'][:]-1
        s4 = f.variables['src04'][:]-1
        w1 = f.variables['wgt01'][:]
        w2 = f.variables['wgt02'][:]
        w3 = f.variables['wgt03'][:]
        w4 = f.variables['wgt04'][:]
       
    NO = odata.size   # number of operational grid points
    NN = s1.size      # number of NEMO grid points
    
    # Build matrix
    n = np.array([x for x in range(0,NN)])
    M1 = sp.csr_matrix((w1.flatten(), (n, s1.flatten())), (NN,NO))
    M2 = sp.csr_matrix((w2.flatten(), (n, s2.flatten())), (NN,NO))
    M3 = sp.csr_matrix((w3.flatten(), (n, s3.flatten())), (NN,NO))
    M4 = sp.csr_matrix((w4.flatten(), (n, s4.flatten())), (NN,NO))
    M = M1+M2+M3+M4
    
        # Interpolate by matrix multiply - quite fast
    ndata = M*odata.flatten()

    # Reshape to NEMO shaped array
    ndata=ndata.reshape(s1.shape)
    
    return ndata

def weights_intrp_mtx_gemlam(odata):
    '''POINT: interpolates data from the atmospheric grid onto the ocean grid using a weights matrix | CALLS: | NOTES: this is much faster than interpolating with scipy! | TAKES : one array of size lats 266 x lons 256 of atmospheric data | GIVES: an array of NEMO-Salish size of the new data interpolated | USAGE: | ROLE IN CLUSTER PROJECT: | written by Michael Dunphy, adapted by tjšj, 2017'''
    from IPython import embed
    import netCDF4 as nc
    import scipy.interpolate as spi
    import scipy.sparse as sp
    import numpy as np
    '''takes a matrix of size lon(256), lat(266) and interpolates it onnto the NEMO gird
    size is 266 x 256'''
    weightsfile = '/data/tjarniko/MEOPAR/grid/weights-gem2.5-gemlam_201702.nc'
    with nc.Dataset(weightsfile) as f:
        s1 = f.variables['src01'][:]-1  # minus one for fortran-to-python indexing
        s2 = f.variables['src02'][:]-1
        s3 = f.variables['src03'][:]-1
        s4 = f.variables['src04'][:]-1
        w1 = f.variables['wgt01'][:]
        w2 = f.variables['wgt02'][:]
        w3 = f.variables['wgt03'][:]
        w4 = f.variables['wgt04'][:]
       
    NO = odata.size   # number of operational grid points
    NN = s1.size      # number of NEMO grid points
    
    # Build matrix
    n = np.array([x for x in range(0,NN)])
    M1 = sp.csr_matrix((w1.flatten(), (n, s1.flatten())), (NN,NO))
    M2 = sp.csr_matrix((w2.flatten(), (n, s2.flatten())), (NN,NO))
    M3 = sp.csr_matrix((w3.flatten(), (n, s3.flatten())), (NN,NO))
    M4 = sp.csr_matrix((w4.flatten(), (n, s4.flatten())), (NN,NO))
    M = M1+M2+M3+M4
    
        # Interpolate by matrix multiply - quite fast
    ndata = M*odata.flatten()

    # Reshape to NEMO shaped array
    ndata=ndata.reshape(s1.shape)
    
    return ndata

In [46]:
import arrow
import numpy as np
import glob



def get_windar(start,end):
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    thres = arrow.get('2014-09-12')

    arrow_array = []
    ncar = []

    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    #print(dayslen)
    ts = np.zeros(dayslen)
    #print(arrow_array)

    # ops_y2014m09d12.nc

    for i in range(0,dayslen):

        tdate = arrow_array[i][0]
        yy = tdate.format('YYYY')
        mm = tdate.format('MM')
        dd = tdate.format('DD')
        ymd = f'y{yy}m{mm}d{dd}'

        if tdate < thres:
            dirstring = '/results/forcing/atmospheric/GEM2.5/gemlam/'
            guess = 'gemlam_'+ ymd +'.nc' 
        else:
            dirstring = '/results/forcing/atmospheric/GEM2.5/operational/'
            guess = 'ops_'+ ymd +'.nc'

        #tdir = '/data/tjarniko/results/RIV_PIL/BASE/ncs/'

        w = glob.glob(dirstring+guess)
        w = w[0]
        ncar.append(w)
    
    return ncar

start ='2013-01-01'; end ='2013-12-31'
ncar_2013 = get_windar(start,end)
start ='2014-01-01'; end ='2014-12-31'
ncar_2014 = get_windar(start,end)
start ='2015-01-01'; end ='2015-12-31'
ncar_2015 = get_windar(start,end)
start ='2016-01-01'; end ='2016-12-31'
ncar_2016 = get_windar(start,end)

In [47]:
def produce_interpolated_wind_netcdf(start, end):
    '''POINT: produces a netcdf file containing SalishSeaNEMO sized arrays of wind magnitude, wind stress, and wind energy for each in a given series of days | CALLS: create_gemlist, weights_intrp_mtx (both from wind_fxn) | NOTES: day numbering starts at 0! the fname is hardcoded to a given directory into which nc files are deposited, could be changed maybe | TAKES : year, day start (starts at 0), day end  | GIVES: netcdfs in a given directory| USAGE: produce_interpolated_wind_netcdf | ROLE IN CLUSTER PROJECT: | written 2017, tjšj '''
    
    import netCDF4 as nc
    import xarray as xr
    
    ncar = get_windar(start,end)
    
    arrow_array = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)
    
    for i in range(0, len(ncar)):
        if i%5 == 0:
            print(i)
        wind = nc.Dataset(ncar[i])
        #print(wind)
        u_wind = wind['u_wind'][:]
        v_wind = wind['v_wind'][:]

        wind_mag = np.sqrt((u_wind)*(u_wind)+(v_wind)*(v_wind))
        wind_stress = wind_mag ** 2
        wind_energy = wind_mag ** 3

        daily_wm = np.average(wind_mag, axis=0)
        daily_ws = np.average(wind_stress, axis=0)
        daily_we = np.average(wind_energy, axis=0)
        
        tdate = arrow_array[i][0]
        ymd = tdate.format('YYYYMMDD')
        
        thres = arrow.get('2014-09-12')
        if tdate<thres:
            daily_wm_int = weights_intrp_mtx_gemlam(daily_wm)
            daily_ws_int = weights_intrp_mtx_gemlam(daily_ws)
            daily_we_int = weights_intrp_mtx_gemlam(daily_we)
        
        else:
        
            daily_wm_int = weights_intrp_mtx_ops(daily_wm)
            daily_ws_int = weights_intrp_mtx_ops(daily_ws)
            daily_we_int = weights_intrp_mtx_ops(daily_we)  
            
        windthings = xr.Dataset({'daily_avg_windmag': (['y','x'], daily_wm_int),
                                 'daily_avg_windstress': (['y','x'], daily_ws_int),
                                 'daily_avg_windenergy': (['y','x'], daily_we_int),})
        
        fname = f'./WINDFILES_interp/windint_{ymd}.nc' 
        
        windthings.to_netcdf(fname)  
start ='2013-01-01'; end ='2013-12-31'
produce_interpolated_wind_netcdf(start,end)

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360


In [48]:
# start ='2014-01-01'; end ='2014-12-31'
# produce_interpolated_wind_netcdf(start,end)
# start ='2015-01-01'; end ='2015-12-31'
# produce_interpolated_wind_netcdf(start,end)
# start ='2016-01-01'; end ='2016-12-31'
# produce_interpolated_wind_netcdf(start,end)

In [56]:
def yearwinds_de(spacing,stn_b,stn_e,year):
    '''POINT: for a given set of stations, produces 3 timeseries of wind forcing data (magnitude, stress, energy)  station as lists in a single netcdf file (one netcdf file per station) | CALLS: import_bathy, make_stns, filster_station_in_domain (from map_fxn), reads in netcdf files produced by produce_interpolated_wind_netcdf | NOTES: days start at 0, stations start at 0, for a spacing of 10 there should be 580 stations, the start and end station option is so that it can be run in parallel if necessary, directory into which nc files are deposited is hardcoded under stn_name | TAKES : spacing (has been decided as 10 for this project), year, start station, end station | GIVES: netcdf files that contain 3 windrelated timeseries for each station, in a given repo | USAGE: yearwinds_de(10,1,580,2016) | ROLE IN CLUSTER PROJECT: | written 2017, tjšj '''
    import sys
    sys.path.append('./extraction_scripts')
    
    import map_fxn as mf
    import time
    import xarray as xr
    import numpy as np

    print('Spacing between stations: ' + str(spacing))
    
    noday = 365
    if year == 2016:
        noday = 366
    
    print('no. days, from yearwinds_de fxn')
    print(noday)
    
    bath = '/results/nowcast-sys/NEMO-forcing/grid/mesh_mask_SalishSea2.nc'
    grid = mf.import_bathy(bath)
    fmask = (grid.fmask[0,0,:,:])
    
    stn_x, stn_y = mf.make_stns(spacing)
    d_stn_x, d_stn_y = mf.filter_stn_in_domain(stn_x,stn_y,fmask)
    print('number of stns: ' + str(len(d_stn_x)))
    
    for stn in range(stn_b,stn_e):
    
        print('station is: ' ,str(stn))
        print('x is :', d_stn_x[stn])
        print('y is :', d_stn_y[stn])
        
        ts_x = d_stn_x[stn]
        ts_y = d_stn_y[stn]
        
        daily_windmag = np.zeros(noday)
        daily_windstress = np.zeros(noday)
        daily_windenergy = np.zeros(noday)
        
        end_day = noday
        
        start = f'{str(year)}-01-01'
        start = f'{str(year)}-12-31'

        start_run = arrow.get(start)
        end_run = arrow.get(end)

        arrow_array = []
        ncar = []

        for r in arrow.Arrow.span_range('day', start_run, end_run):
            arrow_array.append(r)

        dayslen = len(arrow_array)
        #print(dayslen)
        ts = np.zeros(dayslen)
        #print(arrow_array)

        # ops_y2014m09d12.nc

        for i in range(0,dayslen):

            tdate = arrow_array[i][0]
            yy = tdate.format('YYYYMMDD')

        for day in range(0,end_day):
            print(day)
            ##open the winds file
            #dayfile
            
            filestart = f'./WINDFILES_interp/windint_{yy}.nc'
            
            #print(day)
            #print(filestart) 
            wf = xr.open_dataset(filestart)
            ## extract wm, ws, we for the station for the day
            ## attach it to the corresponding array above
            ## save the resulting 3 lists to a netcdf file as below
            
            #avg_daily_windmag_windstress_energy
            
            daily_we = wf.daily_avg_windenergy[ts_y,ts_x]
            daily_we = daily_we.values
            daily_ws = wf.daily_avg_windstress[ts_y,ts_x]
            daily_ws = daily_ws.values
            daily_wm = wf.daily_avg_windmag[ts_y,ts_x]
            daily_wm = daily_wm.values
            ##be gentle about OB1 errors!!!!
            

            
            daily_windmag[day] = daily_wm
            daily_windstress[day] = daily_ws
            daily_windenergy[day] = daily_we
            
            
            winds = xr.Dataset({'wind_mags':(['t'], daily_windmag), 'wind_stresses':(['t'], daily_windstress), 
                                'wind_energy':(['t'], daily_windenergy)})
            stn_name = './NC_HINDCAST/' + str(year) + '/WIND_TS/stn_' + str(stn) + '_wind_data_sp' + str(spacing) + '.nc' 
            winds.to_netcdf(stn_name)
            

In [ ]:
yearwinds_de(10,0,580,2013)

Spacing between stations: 10
no. days, from yearwinds_de fxn
365
number of stns: 580
station is:  0
x is : 2
y is : 392
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
24

325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
station is:  6
x is : 2
y is : 452
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228

301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
station is:  12
x is : 12
y is : 422
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
2

290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
station is:  18
x is : 22
y is : 402
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
1

267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
station is:  24
x is : 32
y is : 382
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
1

...

242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
station is:  486
x is : 292
y is : 372
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144

218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
station is:  492
x is : 292
y is : 432
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120

199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
station is:  498
x is : 292
y is : 492
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101

172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
station is:  504
x is : 302
y is : 212
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
6

149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
station is:  510
x is : 302
y is : 382
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35

In [55]:
w = nc.Dataset('/data/tjarniko/MEOPAR/analysis_tereza/notebooks/CLUSTER_PAPER/CLEAN/NC_hindcast/2013/WIND_TS/stn_0_wind_data_sp10.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/data/tjarniko/MEOPAR/analysis_tereza/notebooks/CLUSTER_PAPER/CLEAN/NC_hindcast/2013/WIND_TS/stn_0_wind_data_sp10.nc'